In [ ]:
%load_ext dotenv
%dotenv

import pandas as pd
import numpy as np
import logging

#from solarityPlants import Planta, Inversor, Portafolio
from database.database import SolarityDB

from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt

from pvlib import location, temperature, modelchain, pvsystem
from utils.data import module_degradation_factor, meteomatics_meteo_data, get_losses, get_inverter_parameters, db_module_data


logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
PLANTS_STRINGS_QUERY_TEMPLATE = """
                                    SELECT p.nombre AS nombre_planta, numeroSerie, mppt, string, inclinacion, azimuth, p.latitud, 
                                    p.longitud, sp.orientacion, cantidad, potenciaPanel, potenciaString, modeloCecPanel, montaje 
                                    FROM stringPaneles sp 
                                    INNER JOIN equipo e ON e.id = sp.id_equipo 
                                    INNER JOIN planta p ON p.id = e.id_planta
                                """

PLANTS_DEVICES_QUERY_TEMPLATE = """
                                    SELECT p.nombre AS nombre_planta, e.nombre AS nombre_equipo, e.id AS id_equipo, e.modelo, e.numeroSerie, 
                                    factorCalibracionPotTeorica, modeloCecInversor, ce.valor as 'potencia_ac'
                                    FROM equipo e  
                                    INNER JOIN planta p ON p.id = e.id_planta
                                    INNER JOIN tipoEquipo te ON te.id = e.id_tipo
                                    LEFT JOIN caracteristicaEquipo ce ON ce.id_equipo = e.id 
                                    WHERE te.tipo IN ('Solar Inverters', 'SUN2000')
                                    AND (ce.id_caracteristica = 4 or ce.id_caracteristica IS NULL)
                                    AND e.activo = 1
                                """


solarityDB = SolarityDB()
all_plants_strings_df = solarityDB.query_to_df(PLANTS_STRINGS_QUERY_TEMPLATE)
all_plants_devices_df = solarityDB.query_to_df(PLANTS_DEVICES_QUERY_TEMPLATE)

In [ ]:
#all_plants_strings_df = pd.read_excel("all_plants_strings_df.xlsx")
#all_plants_devices_df = pd.read_excel("all_plants_devices_df.xlsx")
plants_df = pd.read_excel("plants_df.xlsx"); plants_df.set_index('nombre', inplace=True)
weather_data = pd.read_csv("weather_data.csv"); weather_data.set_index('period_end', inplace=True)

temperature_parameters = {'Rooftop': temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['insulated_back_glass_polymer'],
                          'Ground mount': temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_polymer']}

In [ ]:
class PlantModel:

    def __init__(self, plant_name, weather_df):
        self.plant_name = plant_name
        self.plant_id = int(plants_df.loc[self.plant_name].id)
        self._set_weather(weather_df)
        self.inverters = self._create_model().copy()
    
    def _set_weather(self, weather_df):
        self.weather = weather_df.copy()
        self.module_degradation = [1]*int(len(weather_df.index)*2)


    def _create_model(self):
        ## Datos de la planta y fechas
        plant_data = plants_df.loc[self.plant_name]
        latitud, longitud = plant_data['latitud'], plant_data['longitud']

        site_location = location.Location(latitud, longitud)
        plant_losses = get_losses(plant_name=self.plant_name)


        ################### DATOS DE STRINGS ################
        # Filtrar strings y dispositivos de la planta
        plant_strings = all_plants_strings_df[all_plants_strings_df['nombre_planta'] == self.plant_name]
        plant_devices = all_plants_devices_df[all_plants_devices_df['nombre_planta'] == self.plant_name]

        if plant_strings.empty:
            logging.warning(f"{self.plant_name} sin datos de STRINGS, abortando...")


        inverters_models = []

        print(plant_strings)

        for _, device_data in plant_devices.iterrows():
            serial = device_data.numeroSerie
            id_equipo = device_data.id_equipo

            inverter_ac_power = device_data.potencia_ac
            inverter_model = device_data.modeloCecInversor

            inverter_strings = plant_strings[plant_strings.numeroSerie == serial]

            if 'Not assigned' in list(inverter_strings.modeloCecPanel):
                logging.warning(f"{self.plant_name} sin datos de modelo de modulo")
                logging.warning("Abortando...")
                full_iteration = False
                break

            ############### CREACION DE ARRAYS ###############################
            arrays = [pvsystem.Array(mount=pvsystem.FixedMount(surface_tilt=array_data[1].inclinacion,
                                                                surface_azimuth=array_data[1].azimuth),
                                        module_parameters=db_module_data(array_data[1].modeloCecPanel),
                                        temperature_model_parameters=temperature_parameters[array_data[1].montaje],
                                        strings=1,
                                        modules_per_string=array_data[1].cantidad)
                        for array_data in inverter_strings.iterrows()]


            if not arrays:
                full_iteration = False
                logging.warning(f"{self.plant_name} sin datos de arrays, equipo SN: {serial}")
                logging.warning("Abortando...")
                break

            # Creacion de sistema compuesto por un inversor
            ac_model, inverter_data = get_inverter_parameters(inverter_model_name=inverter_model,  inverter_ac_power=inverter_ac_power)
            inverter_system = pvsystem.PVSystem(arrays=arrays, inverter_parameters=inverter_data)

            # Asignacion de perdidas para modelo
            for key in plant_losses.keys():
                inverter_system.losses_parameters[key] = plant_losses[key]

            mc = modelchain.ModelChain(inverter_system, site_location,
                                        aoi_model='physical',
                                        spectral_model='no_loss',
                                        ac_model=ac_model,
                                        losses_model='pvwatts',
                                        name=f'{id_equipo}')
            
            inverters_models.append(mc)
        

            mc.run_model(weather=self.weather, module_degradation=self.module_degradation)

            # Concatenacion de datos de potencia
            inverter_power = pd.DataFrame(mc.results.ac.copy())
            inverter_power['id_equipo'] = int(mc.name)

            plt.plot(mc.results.ac)
            plt.show()
        


        return inverters_models
    
    
    def inverters_output(self, weather_df, single_inverter=False):
        inverters_mc_result = []
        module_degradation = [1]*int(len(weather_df.index)*2)

        for mc in self.inverters:
            ## Ejecutar modelo
            print(mc)
            mc.run_model(weather=weather_df, module_degradation=module_degradation)

            # Concatenacion de datos de potencia
            inverter_power = pd.DataFrame(mc.results.ac.copy())
            inverter_power['id_equipo'] = int(mc.name)
            inverters_mc_result.append(inverter_power)
        
            if single_inverter:
                break

        plt.plot(mc.results.ac)

        

        self.inverters_power_df = pd.concat(inverters_mc_result)
        self.inverters_power_df.rename(columns={'p_mp': 'valorTeoricoModelChain', 0: 'valorTeoricoModelChain'}, inplace=True)

        return self.inverters_power_df
    
    def power_output(self, weather_df, single_inverter=False):
        return self.output_from_inverters(
            self.inverters_output(weather_df, single_inverter=single_inverter))
        
    def output_from_inverters(self, inverters_power):
        self.inverters_power_df.index = pd.DatetimeIndex(self.inverters_power_df.index)
        self.plant_power_result = self.inverters_power_df.groupby(pd.Grouper(freq='30Min')).agg({'valorTeoricoModelChain': 'sum'})
        return self.plant_power_result
    


In [ ]:
weather_df = weather_data[weather_data['plant_id'] == 55]
weather = weather_df.tail(1000)

plant_model = PlantModel("SODIMAC HC ÑUBLE", weather_df=weather)


In [ ]:
weather

In [ ]:
#output = plant_model.power_output(weather_df, single_inverter=True)
output_v2 = plant_model.inverters_output(weather, single_inverter=True)


In [ ]:
import matplotlib.pyplot as plt
plt.plot(output_v2.valorTeoricoModelChain)

In [ ]:

result = plant_model.inverters_output(weather_df)


inverters_power_dfs = result.rename(columns={'p_mp': 'valorTeoricoModelChain', 0: 'valorTeoricoModelChain'})
inverters_power_dfs = inverters_power_dfs[['valorTeoricoModelChain', 'id_equipo']]

# Dejar en 0 valores negativos
inverters_power_dfs.loc[inverters_power_dfs['valorTeoricoModelChain'] < 0, 'valorTeoricoModelChain'] = 0.0
plant_power = inverters_power_dfs.groupby(pd.Grouper(freq='30Min')).agg({'valorTeoricoModelChain': 'sum'})

In [ ]:
plant_power

In [ ]:
from solarityDB import SolarityDB

SOLCAST_DATA_QUERY_TEMPLATE =   """
                                    SELECT period_end, ghi, dni, dhi, cloud_opacity, period FROM lecturaRadiacionSolcast lrs 
                                    INNER JOIN planta p ON p.id=lrs.id_planta
                                    WHERE p.nombre = '{plant_name}'
                                    AND period_end >= '{start_datetime}' 
                                    AND period_end < '{end_datetime}'
                                """

start_time = datetime(2022, 8, 1, 0, 0)
end_time = datetime(2023, 7, 31, 23, 55)

start_datetime = datetime.strftime(start_time, '%Y-%m-%d')
end_datetime = datetime.strftime(end_time, '%Y-%m-%d %H:%M:%S')

solarityDB = SolarityDB()


weather_dataframes = []
for plant_name, plant_data in plants_df.iterrows():

    plant_id = plant_data['id']
    solcast_data_query = SOLCAST_DATA_QUERY_TEMPLATE.format(plant_name=plant_name, start_datetime=start_datetime, end_datetime=end_datetime)
    solcast_data = solarityDB.query_to_df(solcast_data_query)
    solcast_data.set_index('period_end', inplace=True)

    times = pd.DatetimeIndex(solcast_data.index)
    if times.empty:
        logging.warning("Iterando en dataframe de tiempo vacío, ABORTANDO...")
        continue

    # Creacion de dataframe de datos meteorologicos (mezcla entre meteomatics y datos de solcast)
    meteomatics_timeseries = meteomatics_meteo_data(plant_id=plant_id, times=times, solarity_db_conn=solarityDB)
    columns_renaming = {'air_temp': 'temp_air'}
    weather_df = solcast_data.join(meteomatics_timeseries, how='inner')
    weather_df = weather_df.rename(columns=columns_renaming)

    weather_df['plant_id'] = int(plant_id)

    weather_dataframes.append(weather_df.copy())